In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/June2022-June2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Oct2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Aug2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Nov2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Feb2022-Feb2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Apr2022-Apr2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Sep2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Dec2021-Table 1.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Mar2022-Mar2022.csv
/kaggle/input/rental-ca-vacant-market-rentalprices-data/Jan2022-Jan2022.csv
/kaggle/input/housing-affordability-in-canada/Structural-dwellings-household-size.csv
/kaggle/input/housing-affordability-in-canada/population_dwellings_count.csv

Load data

In [26]:
from pathlib import Path
df = pd.read_csv(Path('/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv'))

In [27]:
print(df)

              City   1 BED     M/M    Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,334   2.37%  18.12  $3,324   6.47%  23.71
1          Toronto  $2,065   2.08%  12.96  $2,849   2.63%  17.19
2       Burlington  $2,017   0.00%  15.85  $2,261   0.22%  11.76
3         Oakville  $2,010   5.29%   8.12  $2.333   0.09%   3.37
4        Etobicoke  $1,897   2.49%  10.68  $2.446   4.80%  15.87
5          Burnaby  $1,883  -3.88%  15.24  $2,563   5.39%  14.52
6      Mississauga  $1,817   0.11%   5.27  $2,211   0.32%   7.64
7           Guelph  $1,811   3.37%  12.91  $2,179  -0.23%  16.09
8      Victoria BC  $1.776   0.74%   6.86  $2,736   7.67%    NaN
9           Barrie  $1.771   0.11%   2.02  $2.084   0.82%  -3.43
10         Vaughan  $1,730   6.86%  11.54  $2.338   6.08%  10.44
11      North York  $1,725    1.29   4.10  $2,259   3.62%   8.82
12       Kitchener  $1,701   5.59%  11.39  $2,037  -1.69%  17.34
13      Surrev. BC  $1.655   6.64%  12.89  $2,080    5.96    NaN
14          Oshawa  $1,64

Clean the prices column

In [47]:
import re
p = re.compile(r'\D')

In [56]:
def cleanupandadddate(file_name, month, year):
    df = pd.read_csv(Path(file_name))
    print(df)
    df['1 BED'] = [p.sub('', x) for x in df['1 BED']]
    df['2 BED'] = [p.sub('', x) for x in df['2 BED']]
    try:
        df['M/M'] = [re.sub(r'%', '', x) for x in df['M/M']]
    except TypeError as te:
        print(te)
    try:
        df['Y/Y'] = [re.sub(r'%', '', x) for x in df['Y/Y']]
    except TypeError as te:
        print(te)
    try:
        df['M/M.1'] = [re.sub(r'%', '', x) for x in df['M/M.1']]
    except TypeError as te:
        print(te)
    try:
        df['Y/Y.1'] = [re.sub(r'%', '', x) for x in df['Y/Y.1']]
    except TypeError as te:
        print(te)
    df['Month'] =[month]*df.shape[0]
    df['Year'] = [year]*df.shape[0]
    df.columns = ['City', '1-Bed($)', 'M/M-1-BED(%)', 'Y/Y-1-BED(%)', '2-BED', 'M/M-2-BED(%)', 'Y/Y-2-BED(%)', 'Month', 'Year']
    return df

In [57]:
updf = cleanupandadddate('/kaggle/input/rental-ca-vacant-market-rentalprices-data/May2022-Table 1.csv', 'May', '2022')

              City   1 BED     M/M    Y/Y   2 BED   M/M.1  Y/Y.1
0        Vancouver  $2,334   2.37%  18.12  $3,324   6.47%  23.71
1          Toronto  $2,065   2.08%  12.96  $2,849   2.63%  17.19
2       Burlington  $2,017   0.00%  15.85  $2,261   0.22%  11.76
3         Oakville  $2,010   5.29%   8.12  $2.333   0.09%   3.37
4        Etobicoke  $1,897   2.49%  10.68  $2.446   4.80%  15.87
5          Burnaby  $1,883  -3.88%  15.24  $2,563   5.39%  14.52
6      Mississauga  $1,817   0.11%   5.27  $2,211   0.32%   7.64
7           Guelph  $1,811   3.37%  12.91  $2,179  -0.23%  16.09
8      Victoria BC  $1.776   0.74%   6.86  $2,736   7.67%    NaN
9           Barrie  $1.771   0.11%   2.02  $2.084   0.82%  -3.43
10         Vaughan  $1,730   6.86%  11.54  $2.338   6.08%  10.44
11      North York  $1,725    1.29   4.10  $2,259   3.62%   8.82
12       Kitchener  $1,701   5.59%  11.39  $2,037  -1.69%  17.34
13      Surrev. BC  $1.655   6.64%  12.89  $2,080    5.96    NaN
14          Oshawa  $1,64

In [58]:
print(updf)

              City 1-Bed($) M/M-1-BED(%)  Y/Y-1-BED(%) 2-BED M/M-2-BED(%)  \
0        Vancouver     2334         2.37         18.12  3324         6.47   
1          Toronto     2065         2.08         12.96  2849         2.63   
2       Burlington     2017         0.00         15.85  2261         0.22   
3         Oakville     2010         5.29          8.12  2333         0.09   
4        Etobicoke     1897         2.49         10.68  2446         4.80   
5          Burnaby     1883        -3.88         15.24  2563         5.39   
6      Mississauga     1817         0.11          5.27  2211         0.32   
7           Guelph     1811         3.37         12.91  2179        -0.23   
8      Victoria BC     1776         0.74          6.86  2736         7.67   
9           Barrie     1771         0.11          2.02  2084         0.82   
10         Vaughan     1730         6.86         11.54  2338         6.08   
11      North York     1725         1.29          4.10  2259         3.62   